<a href="https://colab.research.google.com/github/AldoF95/bart-chat-summarizer-finetuning/blob/main/Bart_large_xsum_fine_tuned_samsum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install necessary packages

In [ ]:
!pip install transformers datasets 

In [ ]:
!pip install rouge.score nltk py7zr

# Fine tune with transformers

In [3]:
import transformers
from datasets import load_dataset, load_metric, load_from_disk
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Loading dataset error

Sometimes when loading the dataset wile in GPU enviorment it will give the error that it cannot find the *samsum* dataset. The workaround is to load the dataset while in CPU mode then save it localy or on you drive. After that just switch back to GPU and load the dataset from the local file using *load_from_disk()*

In [37]:
#data = load_dataset('samsum')
#data.save_to_disk('/content/samsum')
data = load_from_disk("/content/drive/MyDrive/samsum")
metric = load_metric('rouge')
model_checkpoints = 'facebook/bart-large-xsum'

## Data tokenization

**max_input** and **max_target** can variy depending on the available computing power

In [ ]:
max_input = 512
max_target = 128
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoints)

In [39]:
def preprocess_data(data_to_process):
  #get the dialogue text
  inputs = [dialogue for dialogue in data_to_process['dialogue']]
  #tokenize text
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)

  #tokenize labels
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['summary'], max_length=max_target, padding='max_length', truncation=True)
    
  model_inputs['labels'] = targets['input_ids']
  #reuturns input_ids, attention_masks, labels
  return model_inputs

In [40]:
tokenize_data = data.map(preprocess_data, batched = True, remove_columns=['id', 'dialogue', 'summary'])

Loading cached processed dataset at /content/drive/MyDrive/samsum/train/cache-721199377c92b892.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

Loading cached processed dataset at /content/drive/MyDrive/samsum/validation/cache-5675e30a0f1fd071.arrow


## If memory problems

- sample data to smaller sizes

In [41]:
#sample the data
train_sample = tokenize_data['train'].shuffle(seed=123).select(range(1000))
validation_sample = tokenize_data['validation'].shuffle(seed=123).select(range(500))
test_sample = tokenize_data['test'].shuffle(seed=123).select(range(200))

Loading cached shuffled indices for dataset at /content/drive/MyDrive/samsum/train/cache-a32e376b6e54fde5.arrow
Loading cached shuffled indices for dataset at /content/drive/MyDrive/samsum/validation/cache-00a901c3938bcb93.arrow


In [25]:
tokenize_data['train'] = train_sample
tokenize_data['validation'] = validation_sample
tokenize_data['test'] = test_sample

In [42]:
tokenize_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

## Training process

In [11]:
#load model
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

Depending on computing power, batch size can go as low as 1 if necessary

In [12]:
batch_size = 1

In [13]:
#collator to create batches. It preprocess data with the given tokenizer
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

In [43]:
#####################
# metrics
# compute rouge for evaluation 
#####################

def compute_rouge(pred):
  predictions, labels = pred
  #decode the predictions
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}

In [44]:
args = transformers.Seq2SeqTrainingArguments(
    'conversation-summ',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size= 1,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    eval_accumulation_steps=1,
    fp16=True
    )
#only CUDA available -> fp16=True

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [45]:
trainer = transformers.Seq2SeqTrainer(
    model, 
    args,
    train_dataset=tokenize_data['train'],
    eval_dataset=tokenize_data['validation'],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

Using amp half precision backend


In [30]:
!nvidia-smi

Tue Apr 12 15:44:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    60W / 149W |   9445MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [35]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 10
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 2
  Total optimization steps = 15


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,0.586245,45.326400,22.358600,37.386900,37.386900,33.600000
2,No log,0.627075,42.663200,18.053400,33.096000,32.832800,32.000000
3,No log,0.606930,42.437500,18.446100,32.710300,32.710300,30.200000


***** Running Evaluation *****
  Num examples = 5
  Batch size = 1
***** Running Evaluation *****
  Num examples = 5
  Batch size = 1
***** Running Evaluation *****
  Num examples = 5
  Batch size = 1


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=15, training_loss=0.03188169797261556, metrics={'train_runtime': 62.5513, 'train_samples_per_second': 0.48, 'train_steps_per_second': 0.24, 'total_flos': 32506569031680.0, 'train_loss': 0.03188169797261556, 'epoch': 3.0})

## Testing the fine tuned model

In [ ]:
conversation = """
Rann: Hey Harry, how have you been? Long time no see!
Harry: Hey! What a surprise! 
Harry: Yes, you are right, we haven’t seen each other in a long time. How have you been?
Rann: There is an important campaign next week which is keeping me busy otherwise rest is going good in my life. 
Rann: How about you?
Harry: Oh! I just finished a meeting with a very important client of mine and now I finally have some free time. I feel relieved that I’m done with it.
Rann: Good for you then. Hey! Let’s make a plan and catch up with each other after next week. 
Rann: What do you say?
Harry: Sure, why not? Give me a call when you are done with your project.
Rann: Sure, then. 
Rann: Bye, take care.
Harry: Bye buddy.
"""

In [ ]:
model_inputs = tokenizer(conversation,  max_length=max_input, padding='max_length', truncation=True)

In [ ]:
model_inputs

{'input_ids': [0, 50118, 500, 2279, 35, 11468, 3268, 6, 141, 33, 47, 57, 116, 2597, 86, 117, 192, 328, 50118, 29345, 35, 11468, 328, 653, 10, 2755, 328, 1437, 50118, 29345, 35, 3216, 6, 47, 32, 235, 6, 52, 2220, 17, 27, 90, 450, 349, 97, 11, 10, 251, 86, 4, 1336, 33, 47, 57, 116, 50118, 500, 2279, 35, 345, 16, 41, 505, 637, 220, 186, 61, 16, 2396, 162, 3610, 3680, 1079, 16, 164, 205, 11, 127, 301, 4, 1437, 50118, 500, 2279, 35, 1336, 59, 47, 116, 50118, 29345, 35, 5534, 328, 38, 95, 1550, 10, 529, 19, 10, 182, 505, 3653, 9, 4318, 8, 122, 38, 1747, 33, 103, 481, 86, 4, 38, 619, 15126, 14, 38, 17, 27, 119, 626, 19, 24, 4, 50118, 500, 2279, 35, 2497, 13, 47, 172, 4, 11468, 328, 2780, 17, 27, 29, 146, 10, 563, 8, 2916, 62, 19, 349, 97, 71, 220, 186, 4, 1437, 50118, 500, 2279, 35, 653, 109, 47, 224, 116, 50118, 29345, 35, 9136, 6, 596, 45, 116, 12192, 162, 10, 486, 77, 47, 32, 626, 19, 110, 695, 4, 50118, 500, 2279, 35, 9136, 6, 172, 4, 1437, 50118, 500, 2279, 35, 36255, 6, 185, 575, 4, 501

In [ ]:
raw_pred, _, _ = trainer.predict([model_inputs])

***** Running Prediction *****
  Num examples = 1
  Batch size = 1


In [ ]:
raw_pred

array([[    2,     0,     0,     0,   500,  2279,     8,  3268,  2220,
           75,   450,   349,    97,    13,    10,   251,    86,     4,
          248,  2279,    34,    41,   505,   637,   220,   186,     4,
         3268,  1550,    10,   529,    19,    10,  3653,     9,    39,
            8,    16, 15126,    14,    37,    18,   626,    19,    24,
            4,  3268,    40,   492,   248,  2279,    10,   486,    77,
           37,    18,  1550,    19,    39,   695,     4,     2]])

In [ ]:
tokenizer.decode(raw_pred[0])

'</s><s>Harry has just finished a meeting with a very important client of his. Rann will</s>'

In [ ]:
tokenizer.decode(raw_pred[0])

"</s><s><s><s>Rann and Harry haven't seen each other for a long time. Rann has an important campaign next week. Harry finished a meeting with a client of his and is relieved that he's done with it. Harry will give Rann a call when he's finished with his project.</s>"